In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_excel("/content/drive/MyDrive/MTP phase 2/DRT_price adjustments_changes_in_rate_calculation.xlsx")
df.head(10)

,Case No.,TEXT,Class
0,22.0,The dispute between the parties relates to the...,1.0
1,22.0,The respondent claims that MTNL was liable to ...,1.0
2,22.0,"According to MTNL, the agreement stood novated...",1.0
3,22.0,"By the arbitral award, the arbitrator has acce...",1.0
4,22.0,The underlying controversy involved in the pre...,1.0
5,22.0,The rates for the said work were discovered to...,1.0
6,22.0,"On 13.05.2003, a meeting was held between MTNL...",1.0
7,22.0,"MTNL, by a letter dated 03.07.2003, informed C...",1.0
8,22.0,The respondent submitted its first bill for ex...,1.0
9,22.0,"However, MTNL cleared the said bill by unilate...",1.0


In [ ]:
df = df.drop(['Case No.'], axis=1)

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
from nltk.corpus import stopwords

In [ ]:
#Except = ['no']
def remove_stopwords(text):
  new_text = []
  for word in text.split():
    if word in stopwords.words('english'):# and word not in Except:
      new_text.append('')
    else:
      new_text.append(word)
  x = new_text[:]
  new_text.clear()
  return " ".join(x)

In [ ]:
new_text = remove_stopwords(df['TEXT'][0])

In [ ]:
new_text

'The dispute   parties relates   price payable  MTNL   work  laying  cables  trench-less technology.'

In [ ]:
df['TEXT'] = df['TEXT'].apply(remove_stopwords)



In [ ]:
df

,TEXT,Class
0,The dispute parties relates price payable ...,1.0
1,The respondent claims MTNL liable pay sai...,1.0
2,"According MTNL, agreement stood novated fa...",1.0
3,"By arbitral award, arbitrator accepted res...",1.0
4,The underlying controversy involved present ...,1.0
...,...,...
252,The Respondent No. 2 - ICA vide Letter dated 0...,0.0
253,The Respondent No. 2 - ICA advised State rec...,0.0
254,The Appellant State refuted objection raised ...,0.0
255,"On 24.09.2015, Respondent No. 1 - M/s. G. F. T...",0.0


In [ ]:
import string

In [ ]:
def text_cleaning(a):
  remove_punctuation = [char for char in a if char not in string.punctuation]
  remove_punctuation = ''.join(remove_punctuation)
  return [word for word in remove_punctuation.split()]

In [ ]:
print(df.iloc[:,0].apply(text_cleaning))

0      [The, dispute, parties, relates, price, payabl...
1      [The, respondent, claims, MTNL, liable, pay, s...
2      [According, MTNL, agreement, stood, novated, f...
3      [By, arbitral, award, arbitrator, accepted, re...
4      [The, underlying, controversy, involved, prese...
                             ...                        
252    [The, Respondent, No, 2, ICA, vide, Letter, da...
253    [The, Respondent, No, 2, ICA, advised, State, ...
254    [The, Appellant, State, refuted, objection, ra...
255    [On, 24092015, Respondent, No, 1, Ms, G, F, To...
256    [Respondent, No, 2, ICA, forwarded, said, obje...
Name: TEXT, Length: 257, dtype: object


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
bow_transformer = CountVectorizer(analyzer = text_cleaning).fit(df['TEXT'])

In [ ]:
title_bow = bow_transformer.transform(df['TEXT'])

In [ ]:
x = title_bow.toarray()

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer().fit(title_bow)
title_tfidf = tfidf_transformer.transform(title_bow)

In [ ]:
from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(title_tfidf, df['Class'], test_size = 0.2, random_state = 0)

In [ ]:
model1 = MultinomialNB().fit(X_train, y_train)
model2 = BernoulliNB().fit(X_train, y_train)
model3 = LinearSVC().fit(X_train, y_train)
model4 = LogisticRegression().fit(X_train, y_train)
model5 = KNeighborsClassifier().fit(X_train, y_train)
model6 = DecisionTreeClassifier().fit(X_train, y_train)

In [ ]:
all_pred1 = model1.predict(X_test)
all_pred2 = model2.predict(X_test)
all_pred3 = model3.predict(X_test)
all_pred4 = model4.predict(X_test)
all_pred5 = model5.predict(X_test)
all_pred6 = model6.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, all_pred3)

array([[24,  2],
       [ 1, 25]])

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
print('MultinomialNB:',accuracy_score(y_test,all_pred1))
print('BernoulliNB:',accuracy_score(y_test,all_pred2))
print('LinearSVC:',accuracy_score(y_test,all_pred3))
print('LogisticRegression:',accuracy_score(y_test,all_pred4))
print('KNeighborsClassifier:',accuracy_score(y_test,all_pred5))
print('DecisionTreeClassifier:',accuracy_score(y_test,all_pred6))

MultinomialNB: 0.6538461538461539
BernoulliNB: 0.75
LinearSVC: 0.9423076923076923
LogisticRegression: 0.6538461538461539
KNeighborsClassifier: 0.8269230769230769
DecisionTreeClassifier: 0.8269230769230769
